The __tangle__ step in literate programming converts text in the specification language to the source language; the source language must be valid to the object language.  [`pidgin.tangle`](tangle.ipynb) transpiles __Markdown__ to valid `IPython` flavored __Python__ using the `IPython` configuration system.

Literate `pidgin` documents are reusable as __Python__ modules, documentation, and tests.

In [1]:
    import ast, IPython, re, functools,traitlets, doctest, textwrap, mistune, types, importnb, typing as t
    if __name__ == '__main__': 
        %reload_ext pidgin
    with importnb.Notebook():
        from pidgin import tangle

## IPython configuration.

In [2]:
    def log_ast(shell, node):
        return IPython.core.interactiveshell.InteractiveShell.transform_ast(shell, node)

In [3]:
    def load_ipython_extension(shell):
        from . import json
        unload_ipython_extension(shell)
        shell.ast_transformers.append(ReturnYieldDisplay())
        shell.transform_ast = types.MethodType(log_ast, shell)

In [4]:
    class ReturnYieldDisplay(ast.NodeTransformer):
        def visit_FunctionDef(self, node): return node
        
        def visit_Return(self, node): 
            shell = IPython.get_ipython()
            if (not hasattr(shell, 'tangle')) or (hasattr(shell, 'tangle') and ip.tangle.return_display):
                return ast.Expr(
                    ast.Call(
                        func=ast.parse('__import__("IPython").display.display', mode='eval').body, 
                        args=node.value.elts if isinstance(node.value, ast.Tuple) else [node.value], 
                        keywords=[]))
            return node
        
        def visit_Expr(self, node):
            if isinstance(node.value, ast.Yield): 
                node = self.visit_Return(node.value)
            return node

`MarkdownImporter` imports __Markdown__ files are source.  By default they recieve a Markdown repr.

A markdown file is a notebook with one cell.

In [5]:
    def unload_ipython_extension(shell):
        from . import json
        shell.transform_ast = types.MethodType(type(shell).transform_ast, shell)
        for i in reversed([i for i, object in enumerate(shell.ast_transformers) if isinstance(object, (ReturnYieldDisplay))]): shell.ast_transformers.pop(i)